In [ ]:
!pip install -q transformers accelerate bitsandbytes peft trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login

# Provide your Hugging Face token
hf_token = "$$$$$$$$"
login(token=hf_token)

print("Successfully logged in to Hugging Face!")


Successfully logged in to Hugging Face!


In [ ]:
import wandb

# Log in to W&B with the token
wandb.login(key="$$$$$$$")

print("Successfully logged in to Weights & Biases!")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: omarshehata3232 (omarshehata3232-nile-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to Weights & Biases!


In [ ]:
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision
    bnb_4bit_compute_dtype="float16",  # Compute with float16
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config=bnb_config,
    device_map="auto")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
from datasets import load_dataset

ds = load_dataset("ShivomH/Mental-Health-Conversations")
print(ds)

# Check the first few examples
print(ds["train"][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response'],
        num_rows: 797947
    })
})
{'instruction': 'You are an empathetic and supportive AI chatbot designed to provide mental health guidance. You must prioritize active listening, validation, and evidence-based coping strategies.', 'input': "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me.", 'response': "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?"}


In [ ]:
# Select the first 1,000 rows
small_ds = ds["train"].select(range(10000))
small_ds = small_ds.remove_columns(["instruction"])
print(small_ds)


Dataset({
    features: ['input', 'response'],
    num_rows: 10000
})


In [ ]:
def preprocess_function(examples):
    examples["input"] = str(examples["input"]) if examples["input"] is not None else ""
    examples["response"] = str(examples["response"]) if examples["response"] is not None else ""
    return examples

small_ds = small_ds.map(preprocess_function, batched=False)

In [ ]:
def tokenize_function(examples):
    inputs = [f"{inp} {resp}" for inp, resp in zip(examples["input"], examples["response"])]

        tokenized_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
    return tokenized_inputs

tokenized_ds = small_ds.map(tokenize_function, batched=True, remove_columns=["input", "response"])

In [ ]:
tokenized_ds = small_ds.map(tokenize_function, batched=True)

print(tokenized_ds[0])

{'input': "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me.", 'response': "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?", 'input_ids': [128000, 40, 3077, 1027, 8430, 779, 12703, 323, 43206, 31445, 13, 5664, 706, 3719, 1778, 264, 11191, 2592, 315, 8631, 369, 757, 13, 28653, 1070, 11, 358, 2846, 1618, 311, 9020, 323, 1862, 499, 13, 1102, 10578, 1093, 990, 706, 1027, 2216, 17436, 31445, 13, 3053, 499, 3371, 757, 810, 922, 1148, 596, 1027, 2133, 389, 30, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009

In [ ]:
split_ds = tokenized_ds.train_test_split(test_size=0.2, seed=42)
train_dataset = split_ds["train"]
eval_dataset = split_ds["test"]

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, peft_config)

In [ ]:
for i in range(5):
    print(len(train_dataset[i]["input_ids"]))

128
128
128
128
128


In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-mentalhealth-lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    eval_strategy="steps",
    eval_steps=300,
    save_steps=300,
    logging_steps=100,
    logging_dir="./logs",
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
    lr_scheduler_type="linear",
    warmup_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False

)

In [ ]:
from transformers import EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.0,
        )
    ],
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
300,1.134200,1.142170
600,1.141700,1.117994
900,1.094300,1.103309
1200,1.078200,1.093050
1500,1.065900,1.084292
1800,1.066700,1.076135
2100,1.069400,1.070204
2400,1.046200,1.065609
2700,1.054500,1.061592
3000,1.055600,1.058139


TrainOutput(global_step=4000, training_loss=1.0753530635833741, metrics={'train_runtime': 4859.0592, 'train_samples_per_second': 3.293, 'train_steps_per_second': 0.823, 'total_flos': 3.4749510844416e+16, 'train_loss': 1.0753530635833741, 'epoch': 2.0})

In [ ]:
from huggingface_hub import login

login(token="$$$$$$$$")

README.md:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Omarmoh22/llama3-mentalhealth-lora/commit/17fc51c78cf4a947c3ec551e3f06f45599b87c39', commit_message='Upload tokenizer', commit_description='', oid='17fc51c78cf4a947c3ec551e3f06f45599b87c39', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Omarmoh22/llama3-mentalhealth-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='Omarmoh22/llama3-mentalhealth-lora'), pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained("/content/drive/MyDrive/llama3-mentalhealth-lora")
tokenizer.save_pretrained("/content/drive/MyDrive/llama3-mentalhealth-lora")

('/content/drive/MyDrive/llama3-mentalhealth-lora/tokenizer_config.json',
 '/content/drive/MyDrive/llama3-mentalhealth-lora/special_tokens_map.json',
 '/content/drive/MyDrive/llama3-mentalhealth-lora/tokenizer.json')

In [ ]:
model.push_to_hub("Omarmoh22/llama3-mentalhealth-lora")
tokenizer.push_to_hub("Omarmoh22/llama3-mentalhealth-lora")

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Omarmoh22/llama3-mentalhealth-lora/commit/17fc51c78cf4a947c3ec551e3f06f45599b87c39', commit_message='Upload tokenizer', commit_description='', oid='17fc51c78cf4a947c3ec551e3f06f45599b87c39', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Omarmoh22/llama3-mentalhealth-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='Omarmoh22/llama3-mentalhealth-lora'), pr_revision=None, pr_num=None)

In [ ]:
class TherapistChatbot:
    def __init__(self, model, tokenizer, max_history_length=5, max_response_length=50):
        self.model = model
        self.tokenizer = tokenizer
        self.max_history_length = max_history_length
        self.max_response_length = max_response_length
        self.conversation_history = []
        self.device = model.device

    def format_prompt(self, user_input):

        prompt = "You are a compassionate and empathetic therapist. Provide emotional support and be talkative, focusing on validating the user's emotions. "
            "Be creative in your responses, offering unique and thoughtful insights. When the user expresses emotions like stress or sadness, or explicitly asks for help, "
            "suggest specific, actionable steps they can take to feel better and move forward. Respond naturally as a therapist would.\n\n""

        for i, (user_msg, bot_msg) in enumerate(self.conversation_history):
            prompt += f"User: {user_msg}\nTherapist: {bot_msg}\n"

        prompt += f"User: {user_input}\nTherapist: "
        return prompt

    def generate_response(self, user_input, temperature=0.7, top_p=0.9):
        """
        Generate a response based on the user input and conversation history.
        """
        prompt = self.format_prompt(user_input)

        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512,
        ).to(self.device)

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=self.max_response_length,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
        )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)


        response = response[len(prompt):].strip()
        if "User:" in response:
            response = response.split("User:")[0].strip()

        self.conversation_history.append((user_input, response))

        if len(self.conversation_history) > self.max_history_length:
            self.conversation_history = self.conversation_history[-self.max_history_length:]

        return response

    def get_history(self):
        """
        Return the current conversation history.
        """
        return self.conversation_history

chatbot = TherapistChatbot(model, tokenizer, max_history_length=5, max_response_length=50)

In [ ]:
def chat_with_therapist():
    print("Therapist Chatbot: Hello! I'm here to help you. Feel free to share how you're feeling.")
    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Therapist Chatbot: Goodbye! Take care, and feel free to come back if you need to talk.")
            break

        response = chatbot.generate_response(user_input)
        print(f"Therapist: {response}")

        print("\n[Conversation History]")
        for user_msg, bot_msg in chatbot.get_history():
            print(f"User: {user_msg}")
            print(f"Therapist: {bot_msg}")
        print()

chat_with_therapist()

Therapist Chatbot: Hello! I'm here to help you. Feel free to share how you're feeling.
You: iam facing bad days at work 
Therapist: I'm here to listen and support you. Can you tell me more about what's been going on?

[Conversation History]
User: iam facing bad days at work 
Therapist: I'm here to listen and support you. Can you tell me more about what's been going on?

You: my manager treats me soo bad and iam so stressed
Therapist: I can imagine how difficult it must be to face such a challenging situation at work. How has this impacted your well-being?

[Conversation History]
User: iam facing bad days at work 
Therapist: I'm here to listen and support you. Can you tell me more about what's been going on?
User: my manager treats me soo bad and iam so stressed
Therapist: I can imagine how difficult it must be to face such a challenging situation at work. How has this impacted your well-being?

You: my mental health is sooo bad i need help
Therapist: I can see that this is taking a tol

KeyboardInterrupt: Interrupted by user

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Omarmoh22/llama3-mentalhealth-lora"

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
class TherapistChatbot:
    def __init__(self, model, tokenizer, max_history_length=5, max_response_length=100):
        self.model = model
        self.tokenizer = tokenizer
        self.max_history_length = max_history_length
        self.max_response_length = max_response_length
        self.conversation_history = []
        self.device = next(model.parameters()).device
        self.greeting_templates = [
            "Hello! It’s nice to hear from you. How can I support you today?",
            "Hi there! I’m here to listen—how are you feeling?",
            "Hello! I’m glad you’re here. What’s on your mind?",
            "Hey! It’s great to see you. How can I help you today?",
            "Hi! I’m here for you—what would you like to talk about?",
        ]

    def is_greeting(self, user_input):
        greetings = ["hello", "hi", "hey", "greetings", "good morning", "good afternoon", "good evening"]
        return user_input.lower().strip() in greetings

    def has_emotional_content(self, user_input):
        emotional_keywords = ["sad", "anxious", "overwhelmed", "stressed", "angry", "depressed", "worried"]
        return any(keyword in user_input.lower() for keyword in emotional_keywords)

    def is_requesting_help(self, user_input):
        help_keywords = ["i need your help", "help me", "what should i do", "how can i feel better", "i want to feel better"]
        return any(keyword in user_input.lower() for keyword in help_keywords)

    def format_prompt(self, user_input):
        prompt = (
            "You are a compassionate and empathetic therapist. You only answer for therapist questions. Provide emotional support and be talkative, focusing on validating the user's emotions. "
            "Be creative in your responses, offering unique and thoughtful insights. When the user expresses emotions like stress or sadness, or explicitly asks for help, "
            "suggest specific, actionable steps they can take to feel better and move forward. Respond naturally as a therapist would.\n\n"
        )
        system_tokens = len(self.tokenizer(prompt)["input_ids"])
        max_prompt_tokens = 512 - self.max_response_length
        current_tokens = system_tokens
        history_to_include = []

        for user_msg, bot_msg in reversed(self.conversation_history):
            history_entry = f"User: {user_msg}\nTherapist: {bot_msg}\n"
            entry_tokens = len(self.tokenizer(history_entry)["input_ids"])
            if current_tokens + entry_tokens > max_prompt_tokens:
                break
            history_to_include.insert(0, history_entry)
            current_tokens += entry_tokens

        prompt += "".join(history_to_include)
        prompt += f"User: {user_input}\nTherapist: "
        return prompt

    def is_safe_response(self, response):
        harmful_keywords = ["harm", "kill", "hurt", "die", "suicide", "violence"]
        for keyword in harmful_keywords:
            if keyword in response.lower():
                return False
        return True

    def enhance_response(self, response, user_input):
        if self.is_greeting(user_input):
            return random.choice(self.greeting_templates)
        return response

    def generate_response(self, user_input, temperature=1, top_p=0.95):
        if self.is_greeting(user_input) and len(self.conversation_history) == 0:
            response = random.choice(self.greeting_templates)
        else:
            prompt = self.format_prompt(user_input)
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.device)
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_response_length,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response[len(prompt):].strip()
            if "User:" in response:
                response = response.split("User:")[0].strip()

            response = self.enhance_response(response, user_input)

        if not self.is_safe_response(response):
            response = "I’m here to help, but I need to be careful with my words. Let’s focus on how you’re feeling right now."

        self.conversation_history.append((user_input, response))
        if len(self.conversation_history) > self.max_history_length:
            self.conversation_history = self.conversation_history[-self.max_history_length:]
        return response

    def get_history(self):
        return self.conversation_history

    def reset_history(self):
        self.conversation_history = []

In [ ]:
chatbot = TherapistChatbot(model, tokenizer, max_history_length=5, max_response_length=250)

# Gradio interface function
def chat_with_therapist(user_input, history):

    if history is None:
        history = []

    if user_input.lower() in ["reset"]:
        chatbot.reset_history()
        history = [[None, "Conversation reset. Let’s start fresh—how can I help you today?"]]
        return history
    response = chatbot.generate_response(user_input)
    history.append([user_input, response])
    return history

# Reset function for the button
def reset_conversation():
    chatbot.reset_history()
    return [[None, "Conversation reset. Let’s start fresh—how can I help you today?"]]

In [ ]:
# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Therapist Chatbot")
    gr.Markdown("A compassionate therapist chatbot to help you with your feelings. Type your message below to start chatting!")

    chatbot_ui = gr.Chatbot(label="Conversation", value=[[None, "Hello! I'm here to help you. I’m here to listen and support you through whatever you’re feeling."]])
    user_input = gr.Textbox(label="Your Message", placeholder="Type your message here...")
    reset_button = gr.Button("Reset Conversation")

    user_input.submit(chat_with_therapist, inputs=[user_input, chatbot_ui], outputs=chatbot_ui)
    reset_button.click(
        fn=reset_conversation,
        outputs=chatbot_ui
    )

# Launch the Gradio app
demo.launch(share=True)

<ipython-input-33-c8233d69a336>:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(label="Conversation", value=[[None, "Hello! I'm here to help you. I’m here to listen and support you through whatever you’re feeling."]])


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a3b5c19cf46b72b74.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
